
# NLP Assignment - Question Classification
This notebook outlines the process for data preprocessing, training, and model creation for Question Classification tasks using pretrained Word2Vec embeddings.


In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
import gensim.downloader as api
import random


## Data Preprocessing
Load and preprocess the data for question classification.


In [2]:

# Question Classification Data
train_data = pd.read_csv('../Data/TREC/Raw/train.csv')  # Replace with your path
test_data = pd.read_csv('../Data/TREC/Raw/test.csv')  # Replace with your path
# Preprocess Data (Tokenization, padding, label encoding)
# Add your preprocessing steps here
train_data['text'] = train_data['text'].str.lower()
test_data['text'] = test_data['text'].str.lower()

train_data = train_data.drop(columns=['label-fine'])
test_data = test_data.drop(columns=['label-fine'])

In [3]:
coarse_label = train_data['label-coarse'].unique() # returns an array([])
coarse_label = list(coarse_label) 
selection = random.sample(coarse_label, 2)

In [4]:
for index, row in train_data.iterrows():
    label_coarse = train_data.loc[index, 'label-coarse']
    if label_coarse in selection:
        train_data.loc[index, 'label-coarse'] = 'OTHERS'


for index, row in test_data.iterrows():
    label_coarse = test_data.loc[index, 'label-coarse']
    if label_coarse in selection:
        test_data.loc[index, 'label-coarse'] = 'OTHERS'

C:\Users\ninja\AppData\Local\Temp\ipykernel_26188\603029549.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'OTHERS' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.loc[index, 'label-coarse'] = 'OTHERS'
C:\Users\ninja\AppData\Local\Temp\ipykernel_26188\603029549.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'OTHERS' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_data.loc[index, 'label-coarse'] = 'OTHERS'


In [5]:
train_data

,label-coarse,text
0,0,how did serfdom develop in and then leave russ...
1,OTHERS,what films featured the character popeye doyle ?
2,0,how can i find a list of celebrities ' real na...
3,OTHERS,what fowl grabs the spotlight after the chines...
4,2,what is the full form of .com ?
...,...,...
5447,OTHERS,what 's the shape of a camel 's spine ?
5448,OTHERS,what type of currency is used in china ?
5449,4,what is the temperature today ?
5450,4,what is the temperature for cooking ?


In [6]:
train_df, dev_df = train_test_split(train_data, test_size = 500, shuffle = True)

path = '../Data/TREC/Processed/'

# Export DataFrame to a CSV file
train_df.to_csv(f'{path}train.csv', index=False)
dev_df.to_csv(f'{path}dev.csv', index=False)
test_data.to_csv(f'{path}test.csv', index=False)


## Loading Pretrained Word2Vec Embeddings
Load the pretrained Word2Vec model.


In [7]:
path = '../Data/TREC/Processed'

training_dev_df = pd.read_csv(f'{path}/dev.csv')
training_df = pd.read_csv(f'{path}/train.csv')
test_df = pd.read_csv(f'{path}/test.csv')
print(set(training_df['label-coarse']))

{'4', 'OTHERS', '0', '3', '2'}


In [8]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

training_df['label-coarse'] = label_encoder.fit_transform(training_df['label-coarse'])
training_dev_df['label-coarse'] = label_encoder.fit_transform(training_dev_df['label-coarse'])
test_df['label-coarse'] = label_encoder.fit_transform(test_df['label-coarse'])


In [9]:
y_train = to_categorical(training_df['label-coarse'])
y_val = to_categorical(training_dev_df['label-coarse'])
y_test = to_categorical(test_df['label-coarse'])

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_df['text'])

X_train_seq = tokenizer.texts_to_sequences(training_df['text'])
X_val_seq = tokenizer.texts_to_sequences(training_dev_df['text'])
X_test_seq = tokenizer.texts_to_sequences(test_df['text'])

MAX_LEN = 50

X_train = pad_sequences(X_train_seq,MAX_LEN)
X_val = pad_sequences(X_val_seq,MAX_LEN)
X_test = pad_sequences(X_test_seq,MAX_LEN)

In [11]:

# Load Pretrained Word2Vec Model
w2v_model = api.load('word2vec-google-news-300')


In [12]:
_, embedding_dim = w2v_model.vectors.shape

In [13]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in w2v_model:
        embedding_vec = w2v_model[word]
        if embedding_vec is not None:
            embedding_matrix[i] = embedding_vec



## Model Building
Create LSTM models for NER and question classification.


In [17]:
# Model for Question Classification
# Define question classification model architecture

model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    input_length=MAX_LEN,  # Length of input sequences
                    trainable=False))  # Set to True if you want to fine-tune embeddings

# Add LSTM layer
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.2))
# Add Dense layer(s)
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=5, activation='softmax'))  # num_classes: number of output classes
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


## Training
Training procedures for both NER and question classification models.


In [18]:
# Training Question Classification Model
NUM_EPOCH = 200
BATCH_SIZE = 64

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data=(X_val, y_val), callbacks= early_stopper, workers= 4)


Epoch 1/200
78/78 [==============================] - 4s 42ms/step - loss: 1.1384 - accuracy: 0.5359 - val_loss: 0.7739 - val_accuracy: 0.7260
Epoch 2/200
78/78 [==============================] - 3s 37ms/step - loss: 0.5980 - accuracy: 0.7910 - val_loss: 0.5545 - val_accuracy: 0.8020
Epoch 3/200
78/78 [==============================] - 3s 39ms/step - loss: 0.4684 - accuracy: 0.8399 - val_loss: 0.5606 - val_accuracy: 0.7940
Epoch 4/200
78/78 [==============================] - 3s 39ms/step - loss: 0.3750 - accuracy: 0.8736 - val_loss: 0.4054 - val_accuracy: 0.8660
Epoch 5/200
78/78 [==============================] - 3s 42ms/step - loss: 0.3250 - accuracy: 0.8893 - val_loss: 0.3843 - val_accuracy: 0.8740
Epoch 6/200
78/78 [==============================] - 3s 42ms/step - loss: 0.2755 - accuracy: 0.9047 - val_loss: 0.3850 - val_accuracy: 0.8680
Epoch 7/200
78/78 [==============================] - 3s 40ms/step - loss: 0.2381 - accuracy: 0.9170 - val_loss: 0.3732 - val_accuracy: 0.8820
Epoch 


## Evaluation
Metrics and evaluation methods for the models.


In [19]:

# Evaluate NER Model
# Add your evaluation code here
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
# Evaluate Question Classification Model
# Add your evaluation code here


16/16 [==============================] - 0s 9ms/step - loss: 0.3650 - accuracy: 0.8840
Test Loss: 0.3650188148021698, Test Accuracy: 0.8840000033378601
